In [9]:
#################################################
# Developed by Baptiste PICARD                  #
# Started the 23th of June 2020                 #
# picard.baptiste@laposte.net                   #
#                                               #
# The aim of this notebook if to provide a      #
# solution for future Sales competition.        #
# https://www.kaggle.com/c/                     #
# competitive-data-science-predict-future-sales #
#################################################
# 
# In this problem, I will try to predict the "item_cnt_day" or "item_cnt_month" in the submission set.
# This ammount correspond to the total amount of products sold in every shop.
#
# Modules to import 

# Utils 
import re # regular expression
import time 
import datetime

# PySpark
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType, StructType
from pyspark.sql.functions import (isnan, when, count, col,
                                   countDistinct, split, regexp_replace, size, udf) # UDF as User Defined Functions
from pyspark.ml.feature import(VectorAssembler, VectorIndexer,
                                OneHotEncoder, StringIndexer)

# Models
from pyspark.ml.classification import LogisticRegression

# Data
train_set = './data/sales_train.csv'
test_set = './data/test.csv'
subs_set = './data/sample_submission.csv'
items = './data/items.csv'
categories = './data/item_categories.csv'
shops = './data/shops.csv'

In [2]:
spark = SparkSession.builder.appName('future-sales').getOrCreate()
# Open the .csv file  
df_train_set = spark.read.csv(train_set, header=True, inferSchema=True) 
df_test_set = spark.read.csv(test_set, header=True, inferSchema=True) 
df_subs_set = spark.read.csv(subs_set, header=True, inferSchema=True) 
df_shops_set = spark.read.csv(shops, header=True, inferSchema=True) 
df_items_set = spark.read.csv(items, header=True, inferSchema=True) 
df_categories_set = spark.read.csv(categories, header=True, inferSchema=True) 

In [3]:
df_subs_set.printSchema()
df_train_set.printSchema()
df_test_set.printSchema()
df_shops_set.printSchema()
df_items_set.printSchema()
df_categories_set.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- item_cnt_month: double (nullable = true)

root
 |-- date: string (nullable = true)
 |-- date_block_num: integer (nullable = true)
 |-- shop_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- item_price: double (nullable = true)
 |-- item_cnt_day: double (nullable = true)

root
 |-- ID: integer (nullable = true)
 |-- shop_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)

root
 |-- shop_name: string (nullable = true)
 |-- shop_id: integer (nullable = true)

root
 |-- item_name: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- item_category_id: string (nullable = true)

root
 |-- item_category_name: string (nullable = true)
 |-- item_category_id: integer (nullable = true)



In [4]:
# Creating the train DataFrame
df_train = df_train_set.join(df_shops_set, on='shop_id')
df_train = df_train.join(df_items_set, on='item_id')
df_train = df_train.join(df_categories_set, on='item_category_id')
df_train = df_train.drop('shop_id', 'item_id', 'item_category_id')
df_train.printSchema()
df_train.show()
print(df_train.count(), "rows in the train dataframe.")

# Creating the test DataFrame
df_sub = df_test_set.join(df_shops_set, on='shop_id')
df_sub = df_sub.join(df_items_set, on='item_id')
df_sub = df_sub.join(df_categories_set, on='item_category_id')
df_sub = df_sub.join(df_subs_set, on='ID')
df_sub = df_sub.drop('shop_id', 'item_id', 'item_category_id')
df_sub.printSchema()
df_sub.show()
print(df_sub.count(), "rows in the submission dataframe.")

root
 |-- date: string (nullable = true)
 |-- date_block_num: integer (nullable = true)
 |-- item_price: double (nullable = true)
 |-- item_cnt_day: double (nullable = true)
 |-- shop_name: string (nullable = true)
 |-- item_name: string (nullable = true)
 |-- item_category_name: string (nullable = true)

+----------+--------------+----------+------------+--------------------+--------------------+--------------------+
|      date|date_block_num|item_price|item_cnt_day|           shop_name|           item_name|  item_category_name|
+----------+--------------+----------+------------+--------------------+--------------------+--------------------+
|02.01.2013|             0|     999.0|         1.0|"Ярославль ТЦ ""А...|   ЯВЛЕНИЕ 2012 (BD)|      Кино - Blu-Ray|
|03.01.2013|             0|     899.0|         1.0|"Москва ТРК ""Атр...|DEEP PURPLE  The ...|      Музыка - Винил|
|05.01.2013|             0|     899.0|        -1.0|"Москва ТРК ""Атр...|DEEP PURPLE  The ...|      Музыка - Винил|
|06

In [5]:
# Data Engineering 
# Nan Values : 
print("NA values")
df_train.select([count(when(isnan(c), c)).alias(c) for c in df_train.columns]).show() 
df_sub.select([count(when(isnan(c), c)).alias(c) for c in df_sub.columns]).show() 
# Count unique values :
print("Distinct values")
df_train.select([countDistinct(c).alias(c) for c in df_train.columns]).show()
df_sub.select([countDistinct(c).alias(c) for c in df_sub.columns]).show()

NA values
+----+--------------+----------+------------+---------+---------+------------------+
|date|date_block_num|item_price|item_cnt_day|shop_name|item_name|item_category_name|
+----+--------------+----------+------------+---------+---------+------------------+
|   0|             0|         0|           0|        0|        0|                 0|
+----+--------------+----------+------------+---------+---------+------------------+

+---+---------+---------+------------------+--------------+
| ID|shop_name|item_name|item_category_name|item_cnt_month|
+---+---------+---------+------------------+--------------+
|  0|        0|        0|                 0|             0|
+---+---------+---------+------------------+--------------+

Distinct values
+----+--------------+----------+------------+---------+---------+------------------+
|date|date_block_num|item_price|item_cnt_day|shop_name|item_name|item_category_name|
+----+--------------+----------+------------+---------+---------+------------

In [6]:
# Unique values : 
shop_name_train = df_train.select("shop_name").distinct().collect()
shop_name_test = df_sub.select("shop_name").distinct().collect()
item_name_train = df_train.select("item_name").distinct().collect()
item_name_test = df_sub.select("item_name").distinct().collect()
item_category_name_train = df_train.select("item_category_name").distinct().collect()
item_category_name_test = df_sub.select("item_category_name").distinct().collect()

In [7]:
shop = []
for item in shop_name_test : 
    if(item not in shop_name_train) : 
        shop.append(item)
name = []
for item in item_name_test : 
    if(item not in item_name_train) : 
        name.append(item)
cat = []
for item in item_category_name_test : 
    if(item not in item_category_name_train) : 
        cat.append(item)
if(len(shop) != 0) :
    print(len(shop), 'Shop item to add \n')
if(len(cat) != 0) :
    print(len(cat), 'Category item to add \n')
if(len(name) != 0) :
    print(len(name), 'items to add \n')

357 items to add 



In [27]:
# Data Engineering on shop names -> Think about using jellyfish librairy ( Jaro Distance ?)
# Spliting the shop_name into city, type and adress
def splitCity(col) :
    l = len(col.split(' '))
    it1 = col.split(' ')[0].replace('"', '') 
    if(l>=3) :
        it2 = col.split(' ')[1].replace('"', '') 
    else : 
        it2 = "Other"
    if(col[0]=='"' and col[-1]=='"')  :
        it3 = re.search(r'"([^;]*)"', col[1:-1]).group(1)[1:-1] 
    else :
        it3 = 'No name'
        
    if(it1 == '!Якутск'):
        it1 = 'Якутск'
    elif(it1 == 'Сергиев'):
        it1 = 'СергиевПосад'
        it2 = 'ТЦ'
    elif(col== 'Интернет-магазин ЧС') :
        it2 = 'Other'
        it3 = 'No ЧС'
    elif(col ==  'Цифровой склад 1С-Онлайн') :
        it2 = 'склад'
        it3 = '1С-Онлайн'
    elif(col ==  'Якутск Орджоникидзе, 56' or col ==  '!Якутск Орджоникидзе, 56 фран') :
        it2 = 'Орджоникидзе'
        it3 = '56'
    elif(col ==  'Жуковский ул. Чкалова 39м²' or col ==  'Жуковский ул. Чкалова 39м?') :
        it1 = 'Жуковский ул.'
        it2 = 'Чкалова'
        it3 = '39м'
    elif(col ==  'Воронеж (Плехановская, 13)') :
        it2 = 'Плехановская'
        it3 = '13'
    elif(col ==  'Выездная Торговля') :
        it2 = 'Торговля'
        it3 = 'No name'
    elif(col ==  'Москва Магазин С21') :
        it2 = 'Магазин'
        it3 = 'С21'
    return str(it1+" "+it2+" "+it3)
    
def getCity(col) :
    return col.split(' ')[0]

def getType(col) :
    return col.split(' ')[1]

def getAdress(col) :
    return col.split(' ')[2]

temp_city = udf(splitCity, StringType()) 
city = udf(getCity, StringType())
typ = udf(getType, StringType())
adress = udf(getAdress, StringType())
df_train = df_train.withColumn("city_temp", temp_city(df_train.shop_name))
df_train = df_train.withColumn("city", city(df_train.city_temp))
df_train = df_train.withColumn("type", typ(df_train.city_temp))
df_train = df_train.withColumn("adress", adress(df_train.city_temp))
df_train = df_train.drop('city_temp')

df_train.select('shop_name', 'city', 'type', 'adress').show(truncate=False)

+--------------------------+---------+----+-------+
|shop_name                 |city     |type|adress |
+--------------------------+---------+----+-------+
|"Ярославль ТЦ ""Альтаир"""|Ярославль|ТЦ  |Альтаир|
|"Москва ТРК ""Атриум"""   |Москва   |ТРК |Атриум |
|"Москва ТРК ""Атриум"""   |Москва   |ТРК |Атриум |
|"Москва ТРК ""Атриум"""   |Москва   |ТРК |Атриум |
|"Москва ТРК ""Атриум"""   |Москва   |ТРК |Атриум |
|"Москва ТРК ""Атриум"""   |Москва   |ТРК |Атриум |
|"Москва ТРК ""Атриум"""   |Москва   |ТРК |Атриум |
|"Москва ТРК ""Атриум"""   |Москва   |ТРК |Атриум |
|"Москва ТРК ""Атриум"""   |Москва   |ТРК |Атриум |
|"Москва ТРК ""Атриум"""   |Москва   |ТРК |Атриум |
|"Москва ТРК ""Атриум"""   |Москва   |ТРК |Атриум |
|"Москва ТРК ""Атриум"""   |Москва   |ТРК |Атриум |
|"Москва ТРК ""Атриум"""   |Москва   |ТРК |Атриум |
|"Москва ТРК ""Атриум"""   |Москва   |ТРК |Атриум |
|"Москва ТРК ""Атриум"""   |Москва   |ТРК |Атриум |
|"Москва ТРК ""Атриум"""   |Москва   |ТРК |Атриум |
|"Москва ТРК

In [30]:
def splitCategory(col) :
    items = {        
        # Cinema
        'Кино - Blu-Ray' : {'field' : 'Cinema', 'category' : 'Blu-ray'},
        'Кино - DVD' : {'field' : 'Cinema', 'category' : 'DVD'},        
        'Кино - Blu-Ray 3D' : {'field' : 'Cinema', 'category' : 'Blu-Ray 3D'},
        'Кино - Коллекционное' : {'field' : 'Cinema', 'category' : 'Collection'}, 
        'Кино - Blu-Ray 4K' : {'field' : 'Cinema', 'category' : 'Blu-Ray 4K'}, 
        # Accessories
        'Аксессуары - PSVita' : {'field' : 'Accessories', 'category' : 'PSVita'},
        'Аксессуары - PS3' : {'field' : 'Accessories', 'category' : 'PS3'},
        'Аксессуары - XBOX 360' : {'field' : 'Accessories', 'category' : 'XBOX 360'},
        'Аксессуары - PSP' : {'field' : 'Accessories', 'category' : 'PSP'},
        'Аксессуары - PS4' : {'field' : 'Accessories', 'category' : 'PS4'},
        'Аксессуары - XBOX ONE' : {'field' : 'Accessories', 'category' : 'XBOX ONE'},
        'PC - Гарнитуры/Наушники' : {'field' : 'Accessories', 'category' : 'Headphones'}, 
        'Аксессуары - PS2' : {'field' : 'Accessories', 'category' : 'PS2'},
        'Элементы питания' : {'field' : 'Accessories', 'category' : 'Battery'},
        # Music
        'Музыка - CD локального производства' : {'field' : 'Music', 'category' : 'CD'},
        'Музыка - Музыкальное видео' : {'field' : 'Music', 'category' : 'Clip video'},
        'Музыка - CD фирменного производства' : {'field' : 'Music', 'category' : 'CD'},
        'Музыка - MP3' : {'field' : 'Music', 'category' : 'MP3'}, 
        'Музыка - Подарочные издания' : {'field' : 'Music', 'category' : 'Gift Edition'},
        'Музыка - Винил' : {'field' : 'Music', 'category' : 'Vinyl'},
        # Games 
        'Игры - PSVita' : {'field' : 'Games', 'category' : 'PSVita'},
        'Игры - XBOX 360' : {'field' : 'Games', 'category' : 'XBOX 360'},
        'Игры - PSP' : {'field' : 'Games', 'category' : 'PSP'},
        'Игры PC - Стандартные издания' : {'field' : 'Games', 'category' : 'Pc Standard Editions'},
        'Игры - PS3' : {'field' : 'Games', 'category' : 'PS3'},
        'Игры PC - Дополнительные издания' : {'field' : 'Games', 'category' : 'Pc DLC'},
        'Игры PC - Коллекционные издания' : {'field' : 'Games', 'category' : 'Pc Collector'},
        'Игры - Аксессуары для игр' : {'field' : 'Games', 'category' : 'Accessories'},
        'Игры - PS4' : {'field' : 'Games', 'category' : 'PS4'}, 
        'Игры - XBOX ONE' : {'field' : 'Games', 'category' : 'XBOX ONE'}, 
        'Игры - PS2' : {'field' : 'Games', 'category' : 'PS2'},
        'Игры PC - Цифра' : {'field' : 'Games', 'category' : 'Digit'},
        'Игры Android - Цифра' : {'field' : 'Games', 'category' : 'Android'}, 
        'Игры MAC - Цифра' : {'field' : 'Games', 'category' : 'MAC'},
        # Gifts
        'Подарки - Гаджеты, роботы, спорт' : {'field' : 'Gifts', 'category' : 'Gadjet'},
        'Подарки - Мягкие игрушки' : {'field' : 'Gifts', 'category' : 'Toys'},        
        'Подарки - Сувениры' : {'field' : 'Gifts', 'category' : 'Souvenirs'},
        'Подарки - Настольные игры' : {'field' : 'Gifts', 'category' : 'Board games'},
        'Подарки - Настольные игры (компактные)' : {'field' : 'Gifts', 'category' : 'Board games'},
        'Подарки - Развитие' : {'field' : 'Gifts', 'category' : 'Développement'},
        'Подарки - Фигурки' : {'field' : 'Gifts', 'category' : 'Figurines'},
        'Подарки - Сувениры (в навеску)' : {'field' : 'Gifts', 'category' : 'Souvenirs'},
        'Подарки - Сумки, Альбомы, Коврики д/мыши' : {'field' : 'Gifts', 'category' : 'Bags, Albums, Mousepads'},
        'Подарки - Атрибутика' : {'field' : 'Gifts', 'category' : 'Attributes'},
        'Подарки - Открытки, наклейки' : {'field' : 'Gifts', 'category' : 'Postcards / Stickers'},
        'Подарки - Сертификаты, услуги' : {'field' : 'Gifts', 'category' : 'Certificates / Services'},
        # Payement Cards
        'Карты оплаты - PSN' : {'field' : 'Payement Cards', 'category' : 'PSN'},
        'Карты оплаты - Live!' : {'field' : 'Payement Cards', 'category' : 'Live!'},
        'Карты оплаты (Кино, Музыка, Игры)' : {'field' : 'Payement Cards', 'category' : 'Cinema / Music / Games'},
        'Карты оплаты - Live! (Цифра)' : {'field' : 'Payement Cards', 'category' : 'Live!'},
        'Карты оплаты - Windows (Цифра)' : {'field' : 'Payement Cards', 'category' : 'Windows'},
        # Programs
        'Программы - Для дома и офиса' : {'field' : 'Programs', 'category' : 'Home/Office'},
        'Программы - 1С:Предприятие 8' : {'field' : 'Programs', 'category' : 'Entreprise'},
        'Служебные' : {'field' : 'Programs', 'category' : 'Office'},
        'Программы - Обучающие' : {'field' : 'Programs', 'category' : 'Education'},
        'Программы - Для дома и офиса (Цифра)' : {'field' : 'Programs', 'category' : 'Home/Office'},
        'Программы - Обучающие (Цифра)' : {'field' : 'Programs', 'category' : 'Education'},
        'Программы - MAC (Цифра)' : {'field' : 'Programs', 'category' : 'MAC'},
        # Console
        'Игровые консоли - XBOX 360' : {'field' : 'Consoles', 'category' : 'XBOX 360'},
        'Игровые консоли - PSP' : {'field' : 'Consoles', 'category' : 'PSP'},  
        'Игровые консоли - PS3' : {'field' : 'Consoles', 'category' : 'PS3'},
        'Игровые консоли - PSVita' : {'field' : 'Consoles', 'category' : 'PSVita'},
        'Игровые консоли - Прочие' : {'field' : 'Consoles', 'category' : 'Other'}, 
        'Игровые консоли - XBOX ONE' : {'field' : 'Consoles', 'category' : 'XBOX ONE'},
        'Игровые консоли - PS4' : {'field' : 'Consoles', 'category' : 'PS4'},
        'Игровые консоли - PS2' : {'field' : 'Consoles', 'category' : 'PS2'},
        # Books
        'Книги - Методические материалы 1С' : {'field' : 'Books', 'category' : 'Methods'},
        'Книги - Аудиокниги' : {'field' : 'Books', 'category' : 'Audiobooks'},
        'Книги - Аудиокниги 1С' : {'field' : 'Books', 'category' : 'Audiobooks'},
        'Книги - Комиксы, манга' : {'field' : 'Books', 'category' : 'Manga / Comics'},
        'Книги - Артбуки, энциклопедии' : {'field' : 'Books', 'category' : 'Artbooks / Encyclopedias'},
        'Книги - Бизнес литература' : {'field' : 'Books', 'category' : 'Business Literature'},
        'Книги - Путеводители' : {'field' : 'Books', 'category' : 'Guides'},
        'Книги - Художественная литература' : {'field' : 'Books', 'category' : 'Fiction'},
        'Книги - Открытки' : {'field' : 'Books', 'category' : 'Postcards'},
        'Книги - Компьютерная литература' : {'field' : 'Books', 'category' : 'Computer literature'}, 
        'Книги - Познавательная литература' : {'field' : 'Books', 'category' : 'Education'}, 
        'Книги - Цифра' : {'field' : 'Books', 'category' : 'Digit'},
        'Книги - Аудиокниги (Цифра)' : {'field' : 'Books', 'category' : 'Audiobooks'},
        # Services
        'Билеты (Цифра)' : {'field' : 'Services', 'category' : 'Tickets'},
        'Служебные - Билеты' : {'field' : 'Services', 'category' : 'Tickets'},
        'Доставка товара' : {'field' : 'Services', 'category' : 'Delivery'},
        # Media
        'Чистые носители (штучные)' : {'field' : 'Media', 'category' : 'Piece'},
        'Чистые носители (шпиль)' : {'field' : 'Media', 'category' : 'Spire'},
            }
    it1 = items[col]['field']
    it2 = items[col]['category']
    return str(it1+" "+it2)

def getField(col) :
    return col.split(' ')[0]

def getCategory(col) :
    return col.split(' ')[1]

temp_cat = udf(splitCategory, StringType()) 
field = udf(getField, StringType())
category = udf(getCategory, StringType())
df_train = df_train.withColumn("cat_temp", temp_cat(df_train.item_category_name))
df_train = df_train.withColumn("field", field(df_train.cat_temp))
df_train = df_train.withColumn("category", category(df_train.cat_temp))
df_train = df_train.drop('cat_temp')

df_train.select('item_category_name', 'field', 'category').show(truncate=False)

+-----------------------------------+----------------+------+--------+
|item_category_name                 |cat_temp        |field |category|
+-----------------------------------+----------------+------+--------+
|Кино - Blu-Ray                     |Cinema Blu-ray  |Cinema|Blu-ray |
|Музыка - Винил                     |Music Vinyl     |Music |Vinyl   |
|Музыка - Винил                     |Music Vinyl     |Music |Vinyl   |
|Музыка - Винил                     |Music Vinyl     |Music |Vinyl   |
|Музыка - CD фирменного производства|Music CD        |Music |CD      |
|Музыка - Музыкальное видео         |Music Clip video|Music |Clip    |
|Музыка - CD фирменного производства|Music CD        |Music |CD      |
|Музыка - CD локального производства|Music CD        |Music |CD      |
|Музыка - CD локального производства|Music CD        |Music |CD      |
|Музыка - CD локального производства|Music CD        |Music |CD      |
|Музыка - CD локального производства|Music CD        |Music |CD      |
|Музык

In [31]:
df_train.select([countDistinct(c).alias(c) for c in df_train.columns]).show()

+----+--------------+----------+------------+---------+---------+------------------+---------+----+----+------+--------+-----+--------+
|date|date_block_num|item_price|item_cnt_day|shop_name|item_name|item_category_name|city_temp|city|type|adress|cat_temp|field|category|
+----+--------------+----------+------------+---------+---------+------------------+---------+----+----+------+--------+-----+--------+
|1034|            34|     19919|         193|       60|    21657|                84|       56|  31|  13|    47|      75|   11|      50|
+----+--------------+----------+------------+---------+---------+------------------+---------+----+----+------+--------+-----+--------+



In [ ]:
# Unique values : 
categories = df_train.select("category").distinct().collect()
fields = df_train.select("field").distinct().collect()
adresses = df_train.select("adress").distinct().collect()
types = df_train.select("type").distinct().collect()
cities = df_train.select("city").distinct().collect()